In [6]:
#from plotnine import *
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go


px.defaults.template = "ggplot2"

roster_210 = pd.read_csv('./210_roster.csv')
n_students = roster_210.Number.value_counts()

enroll_factor = (n_students[210]+n_students[211])/n_students[211]

n_students[211]

339

In [7]:
fancy_name = lambda x: "Dynamics" if x == 'dyn' else ('Solid Mechanics' if x == 'sol' else 'Statics')

## 

In [8]:
start = {
    'sta': pd.read_excel('./ief/sta.xlsx'),
    'dyn': pd.read_excel('./ief/dyn.xlsx'),
    'sol': pd.read_excel('./ief/sol.xlsx')
}

end = {
    'sta': pd.read_csv('./ief/esta.csv'),
    'dyn': pd.read_csv('./ief/edyn.csv'),
    'sol': pd.read_csv('./ief/esol.csv')
}

sdfs = {}

for class_ in ['sta', 'dyn', 'sol']:
    a = pd.DataFrame(start[class_], columns=['Course resources [I make use of the recommended textbook]', 'Course resources [I make use of the course online reference pages]',
    "Course online reference page [The content in this class stimulates my curiosity about real world problems]",
    "Course online reference page [The reference pages help me understand how concepts in the course connect to each other.]",
    "Course online reference page [The reference pages help me understand how the course content relates to my role as a future engineer.]"])

    a['textbook'] = a['Course resources [I make use of the recommended textbook]']
    a = a.drop('Course resources [I make use of the recommended textbook]', axis=1)

    a['ref'] = a['Course resources [I make use of the course online reference pages]']
    a = a.drop('Course resources [I make use of the course online reference pages]', axis=1)

    a['real-world'] = a["Course online reference page [The content in this class stimulates my curiosity about real world problems]"]
    a = a.drop("Course online reference page [The content in this class stimulates my curiosity about real world problems]", axis=1)

    a['connection'] = a["Course online reference page [The reference pages help me understand how concepts in the course connect to each other.]"]
    a = a.drop("Course online reference page [The reference pages help me understand how concepts in the course connect to each other.]", axis=1)

    a['relate'] = a["Course online reference page [The reference pages help me understand how the course content relates to my role as a future engineer.]"]
    a = a.drop("Course online reference page [The reference pages help me understand how the course content relates to my role as a future engineer.]", axis=1)

    for col in a.columns:
        a[col] = a[col].map(lambda x: int(str(x)[0]) if not pd.isna(x) else None)

    a['textbook'] = a['textbook'].map(lambda x: int(str(x)[0]) if not pd.isna(x) else None)
    a['ref'] = a['ref'].map(lambda x: int(str(x)[0]) if not pd.isna(x) else None)

    sdfs[class_] = a

edfs = {}

for class_ in ['sta', 'dyn', 'sol']:
    a = pd.DataFrame(end[class_], columns=["Course online reference pages [I use the reference pages of this course]","Course online reference pages [The reference pages stimulate my curiosity about real world problems]",
    "Course online reference pages [The reference pages help me understand how concepts in the course connect to each other.]",
    "Course online reference pages [The reference pages help me understand how the course content relates to my role as a future engineer.]"])

    a['ref'] = a["Course online reference pages [I use the reference pages of this course]"]
    a = a.drop("Course online reference pages [I use the reference pages of this course]", axis=1)
    
    a['real-world'] = a["Course online reference pages [The reference pages stimulate my curiosity about real world problems]"]
    a = a.drop("Course online reference pages [The reference pages stimulate my curiosity about real world problems]", axis=1)

    a['connection'] = a["Course online reference pages [The reference pages help me understand how concepts in the course connect to each other.]"]
    a = a.drop("Course online reference pages [The reference pages help me understand how concepts in the course connect to each other.]", axis=1)

    a['relate'] = a["Course online reference pages [The reference pages help me understand how the course content relates to my role as a future engineer.]"]
    a = a.drop("Course online reference pages [The reference pages help me understand how the course content relates to my role as a future engineer.]", axis=1)
    
    for col in a.columns:
        a[col] = a[col].map(lambda x: int(str(x)[0]) if not pd.isna(x) else None)
    

    edfs[class_] = a

In [9]:
col = start['dyn'].columns.to_numpy()

for c in col:
    if 'ref' in c:
        print(c)

Instructor and lectures [The instructor regularly shows/mentions reference pages in lecture]
Course resources [I make use of the course online reference pages]
PrairieLearn Homework [I use the online reference pages to solve problems]
Quizzes [I use the online reference pages to solve problems]
Discussion Sections [I use the online reference pages to solve problems]
Course online reference page [The content in this class stimulates my curiosity about real world problems]
Course online reference page [The reference pages help me understand how concepts in the course connect to each other.]
Course online reference page [The reference pages help me understand how the course content relates to my role as a future engineer.]


In [10]:
sdfs['sol']['relate']

0      4
1      3
2      5
3      2
4      4
      ..
124    3
125    4
126    3
127    5
128    4
Name: relate, Length: 129, dtype: int64

In [11]:
for class_ in ['sta', 'dyn', 'sol']:
    color_map = {1:'#3b76ec',2:'#fe4215',3:'#ffb000',4:'#12ad1c', 5:'#b100b1'}
    fig = go.Figure()
    if class_ == 'dyn': continue
    print(list([sdfs[class_]['textbook'].value_counts()[i+1] for i in range(5)]))
    fig.add_trace(go.Bar(
            x= [1, 2, 3, 4, 5],
            y= list([sdfs[class_]['textbook'].value_counts()[i+1] for i in range(5)]),
            marker=dict(color=[c for k, c in color_map.items()]),
            #marker_line=dict(width=.25,color="black")
        ))
            
    fig.update_layout(
        title=f"",
        yaxis_title="Number of responses",
        showlegend=False,
        height=500,
        width=500,
        bargap=0.3,
        xaxis=dict(
            tickmode = 'array',
            tickvals=[1, 2, 3, 4, 5],
            ticktext=['Strongly Disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly Agree'], 
            range=[0.5, 5.5] , tickfont=dict(size=14)
        ),
        yaxis=dict(
            tickfont=dict(size=14)
        ),
        margin=dict(l=40, r=20, t=20, b=40),plot_bgcolor="#ededed",
        font=dict(size=14)
    )

    fig.update_xaxes(tickangle= -30, 
                      )

    #fig.update_layout(xaxis_title=None)

    fig.write_image(f"./ief/{class_}_textbook.png", scale=3)

    fig.show()


[192, 90, 59, 23, 28]


[78, 24, 22, 1, 4]


In [12]:
for class_ in ['sta', 'dyn', 'sol']:
    color_map = {1:'#3b76ec',2:'#fe4215',3:'#ffb000',4:'#12ad1c', 5:'#b100b1'}

    plot = px.histogram(sdfs[class_],
            x='ref',
            color = 'ref',
            color_discrete_map = color_map)

    tickvals = [len(sdfs[class_])*i/10 for i in range(10)]
    ticktext = [str(i/10) for i in range(10)]


    plot.update_layout(
        title=f"I make use of the course reference pages ({fancy_name(class_)})",
        xaxis = dict(title=f"1: Strongly disagree, 5: Strongly agree", range=[0.5, 5.5]),
        yaxis= dict(title=f"Proportion of responses", range=[0, max(sdfs[class_][sdfs[class_].columns[1]].value_counts())*1.05], tickvals=tickvals, ticktext=ticktext), 
        showlegend=False,
        height=500,
        width=500,
        bargap=0.3
    )

    plot.write_image(f"./ief/start_{class_}_ref.png", scale=3)

    plot.show()

    plot = px.histogram(edfs[class_],
            x='ref',
            color = 'ref',
            color_discrete_map = color_map)

    tickvals = [len(edfs[class_])*i/10 for i in range(10)]
    ticktext = [str(i/10) for i in range(10)]

    plot.update_layout(
        title=f"I make use of the course reference pages ({fancy_name(class_)})",
        xaxis = dict(title=f"1: Strongly disagree, 5: Strongly agree", range=[0.5, 5.5]),
        yaxis= dict(title=f"Proportion of responses", range=[0, max(edfs[class_][edfs[class_].columns[0]].value_counts())*1.05], 
                     tickvals=tickvals, ticktext=ticktext),
        showlegend=False,
        height=500,
        width=500,
        bargap=0.3
    )

    plot.write_image(f"./ief/end_{class_}_ref.png", scale=3)

    plot.show()

In [13]:
import numpy as np

full_question_title = {
    'textbook': 'I make use of the recommended textbook',
    'ref': "I make use of the course online reference pages <br>",
    'real-world': 'The ref. pages stimulate my curiosity about <br> real world problems',
    'connection': 'The ref. pages help me understand how concepts <br> in the course connect to each other',
    'relate': 'The ref. pages help me understand how the course content <br> relates to my role as a future engineer'
}

for col in ['textbook']:#'ref', 'real-world', 'connection', 'relate']:
    for class_ in ['sta', 'dyn', 'sol']:
        if col == 'textbook' and class_ == 'dyn': continue
        color_map = {1:'#3b76ec',2:'#fe4215',3:'#ffb000',4:'#12ad1c', 5:'#b100b1'}
        color_list1 = list([c for k, c in color_map.items()])
        color_list2 = ["#c4d6f9", "#ffc6b9", "#ffe7b3", "#a8f6ad", "#ff9bff"]

        fig = go.Figure()

        y1 = np.zeros(5)
        for i in range(5):
            try:
                y1[i] = sdfs[class_][col].value_counts()[i+1]/sum(sdfs[class_][col].value_counts())
            except KeyError:
                y1[i] = 0

        fig.add_trace(go.Bar(
            x= [1, 2, 3, 4, 5],
            y= y1,
            marker=dict(color=color_list1),
            marker_line=dict(width=.66,color="black")
        ))

        '''y2 = np.zeros(5)
        for i in range(5):
            try:
                y2[i] = edfs[class_][col].value_counts()[i+1]/sum(edfs[class_][col].value_counts())
            except KeyError:
                y2[i] = 0

        fig.add_trace(go.Bar(
            x= [1, 2, 3, 4, 5],
            y= y2,
            marker=dict(color=color_list1)
        ))'''

        fig.update_layout(
        title=f"",
        yaxis_title="Proportion of responses",
        showlegend=False,
        height=500,
        width=500,
        bargap=0.3,
        xaxis=dict(
            tickmode = 'array',
            tickvals=[1, 2, 3, 4, 5],
            ticktext=['Strongly Disagree', 'Disagree', 'Neutral', 'Agree', 'Strongly Agree'], 
            range=[0.5, 5.5] , tickfont=dict(size=14)
        ),
        yaxis=dict(
            tickfont=dict(size=14)
        ),
        margin=dict(l=40, r=20, t=20, b=40),plot_bgcolor="#ededed",
        font=dict(size=14)
    )
        fig.update_xaxes(tickangle= -30, 
                      )
        
        fig.show()
        fig.write_image(f"./ief/{class_}_{col}.png", scale=3)
        continue

In [14]:
list([c for k, c in color_map.items()])

['#3b76ec', '#fe4215', '#ffb000', '#12ad1c', '#b100b1']

In [24]:
import scipy.stats as stats
 
for col in ['ref', 'real-world', 'connection', 'relate']:
    fancy_title = full_question_title[col]
    string = f'{fancy_title.replace("<br>", "")}'
    for class_ in ['sta', 'dyn', 'sol']:
        if col == 'textbook' and class_ == 'dyn': continue
        start_res = sdfs[class_][col]
        end_res = edfs[class_][col]
        #print(p)
        t, p = stats.ttest_rel(end_res[:min(len(start_res), len(end_res))], start_res[:min(len(start_res), len(end_res))], alternative='greater', nan_policy='omit')
        string += f" & {round(p, 5)}" if p > .05 else (f" & \\textbf{{{round(p, 5)}}}" if p > 0.001 else f" & \\textbf{{\\textless 0.001}}")
    string += " \\\\ \n \\hline"
    print(string)

I make use of the course online reference pages  & \textbf{\textless 0.001} & \textbf{\textless 0.001} & \textbf{0.01757} \\ 
 \hline
The ref. pages stimulate my curiosity about  real world problems & 0.96205 & \textbf{0.00127} & 0.21947 \\ 
 \hline
The ref. pages help me understand how concepts  in the course connect to each other & \textbf{\textless 0.001} & \textbf{0.01626} & \textbf{0.00299} \\ 
 \hline
The ref. pages help me understand how the course content  relates to my role as a future engineer & \textbf{\textless 0.001} & \textbf{0.01316} & \textbf{0.00544} \\ 
 \hline


In [12]:
new_sta_ref = sdfs['sta']['ref'].to_numpy()
new_sta_ref = new_sta_ref[~np.isnan(new_sta_ref)]

new_sta_ref = new_sta_ref[~np.isnan(new_sta_ref)]

In [13]:
stats.ttest_ind(edfs['sta']['ref'], new_sta_ref, alternative='greater')

TtestResult(statistic=6.637665534172997, pvalue=3.803487429132523e-11, df=554.0)

In [17]:
percentages = {}

for col in ['ref', 'real-world', 'connection', 'relate']:
    percentages[col] = {}
    for class_ in ['sta', 'dyn', 'sol']:
        percentages[col][class_] = {}
        comb = {
            0: 0,
            1: 0,
            2: 1,
            3: 2,
            4: 2
        }

        y1 = np.zeros(3)
        for i in range(5):
            try:
                y1[comb[i]] += sdfs[class_][col].value_counts()[i+1]/sum(sdfs[class_][col].value_counts())
            except KeyError:
                y1[comb[i]] += 0

        y2 = np.zeros(3)
        for i in range(5):
            try:
                y2[comb[i]] += edfs[class_][col].value_counts()[i+1]/sum(edfs[class_][col].value_counts())
            except KeyError:
                y2[comb[i]] += 0

        percentages[col][class_] = [y2, y1]

In [47]:
from plotly.subplots import make_subplots

red = '#f8766d'
blue = '#619cff'

classes = ['sta', 'dyn', 'sol']
cols = ['ref', 'real-world', 'connection', 'relate']

colors = [red, '#ababab', blue]

small_question_title = {
    'ref': 'Q1',
    "real-world": 'Q2',
    'connection': 'Q3',
    'relate': 'Q4'
}

col_names = [small_question_title[c] for c in cols]
row_names = [fancy_name(c) for c in classes]

legend_names = ['Disagree', 'Neutral', 'Agree']

fig = make_subplots(rows=3, cols=4, row_titles=row_names, column_titles=col_names) 
iteration = 0

annotations = []

for c, col in enumerate(cols):
    for j, class_ in enumerate(classes):
        for i in range(0, 3):
            already_added = []
            for xd, yd in zip(percentages[col][class_], [f'end {class_}', f'start {class_}',]):
                
                fig.add_trace(go.Bar(
                    x=[xd[i]], y=[yd],
                    orientation='h',
                    marker=dict(
                        color=colors[i],
                        line=dict(color='rgb(248, 248, 249)', width=1)
                    ),
                    name=legend_names[i],
                    xaxis='x',
                    showlegend=True if (c == 0 and j == 0 and not iteration % 2) else False,
                ),row=j+1, col=c+1)
                
                #print(xd, True if (c == 0 and j == 0 and not iteration % 2) else False)
                iteration += 1
                fig.update_xaxes(row=j+1, col=c+1, tickvals=[-1], ticktext=['na'])
                fig.update_yaxes(row=j+1, col=c+1, tickvals=[-1], ticktext=['na'])

                if c in [1, 3]:
                    fig.update_yaxes(row=j+1, col=c+1,tickvals=[0, 1], ticktext=['End    ', ' Start   '], tickfont=dict(family='Arial', size=15),)

        fig.update_layout(
            xaxis=dict(
                showgrid=False,
                showline=False,
                showticklabels=False,
                zeroline=False,
                #visible=False
                #domain=[0.15, 0.85]
            ),
            yaxis=dict(
                showgrid=False,
                showline=False,
                showticklabels=False,
                zeroline=False,
            ),
            plot_bgcolor="#ffffff",
            barmode='stack',
            showlegend=True,
            margin=dict(l=75, r=10, t=90, b=50),
            legend=dict(
                yanchor="bottom",
                y=-.1,
                xanchor="left",
                x=0.27,
                orientation="h",
                font=dict(
                    #family="Courier",
                    size=20,
                   # color="black"
                ),
            ),
            title=dict(font=dict(size=50))
        )

        fig.update_xaxes(
            row=1, col=1,
            showgrid=False,
            showline=True,
            showticklabels=True,
            zeroline=False
        )

        fig.update_yaxes(
            row=1, col=1,
            showgrid=False,
            showline=True,
            showticklabels=True,
            zeroline=False
        )

        for yd, xd in zip([f'end {class_}', f'start {class_}'], percentages[col][class_]):
            if c == 0 and j == 0:  # For subplot (1,1)
                xref = 'x1'
                yref = 'y1'
            else:
                xref = f'x{j * len(cols) + c + 1}'
                yref = f'y{j * len(cols) + c + 1}'
                
            # Labeling the first percentage of each bar (x_axis)
            if xd[0] > .15:
                annotations.append(dict(
                    xref=xref,
                    yref=yref,
                    x=xd[0] / 2,
                    y=yd,
                    text=str(round(xd[0] * 100)) + '%',
                    font=dict(family='Arial', size=14, color='rgb(248, 248, 255)'),
                    showarrow=False
                ))
            space = xd[0]
            for i in range(1, len(xd)):
                # Labeling the rest of percentages for each bar (x_axis)
                if xd[i] > .15:
                    annotations.append(dict(
                        xref=f'x{j * len(cols) + c + 1}',
                        yref=f'y{j * len(cols) + c + 1}',
                        x=space + (xd[i] / 2),
                        y=yd,
                        text=str(round(xd[i] * 100)) + '%',
                        font=dict(family='Arial', size=14, color='rgb(248, 248, 255)'),
                        showarrow=False
                    ))
                space += xd[i]    

fig.for_each_annotation(lambda a: a.update(text=f'<b>{a.text}</b>'))

fig.update_layout(annotations=annotations)

## STA REF END 
fig.add_annotation(
    x=sum(percentages['ref']['sta'][1][0:2])+percentages['ref']['sta'][1][-1]/2, y=1,
    text=str(round(percentages['ref']['sta'][1][-1] * 100)) + '%',
    showarrow=False,
    xref="x1", yref="y1",
    font=dict(family='Arial', size=14, color='rgb(248, 248, 255)'),
)

fig.add_annotation(
    x=sum(percentages['ref']['sta'][1][0:1])+percentages['ref']['sta'][1][-2]/2, y=1,
    text=str(round(percentages['ref']['sta'][1][-2] * 100)) + '%',
    showarrow=False,
    xref="x1", yref="y1",
    font=dict(family='Arial', size=14, color='rgb(248, 248, 255)'),
)

fig.add_annotation(
    x=percentages['ref']['sta'][1][0]/2, y=1,
    text=str(round(percentages['ref']['sta'][1][0] * 100)) + '%',
    showarrow=False,
    xref="x1", yref="y1",
    font=dict(family='Arial', size=14, color='rgb(248, 248, 255)'),
)

## STA REF START 
fig.add_annotation(
    x=sum(percentages['ref']['sta'][0][0:2])+percentages['ref']['sta'][0][-1]/2, y=0,
    text=str(round(percentages['ref']['sta'][0][-1] * 100)) + '%',
    showarrow=False,
    xref="x1", yref="y1",
    font=dict(family='Arial', size=14, color='rgb(248, 248, 255)'),
)

fig.add_annotation(
    x=sum(percentages['ref']['sta'][0][0:1])+percentages['ref']['sta'][0][-2]/2, y=0,
    text=str(round(percentages['ref']['sta'][0][-2] * 100)) + '%',
    showarrow=False,
    xref="x1", yref="y1",
    font=dict(family='Arial', size=14, color='rgb(248, 248, 255)'),
)


# DYN REF START
fig.add_annotation(
    x=sum(percentages['ref']['dyn'][1][0:1])+percentages['ref']['dyn'][1][-2]/2, y=1,
    text=str(round(percentages['ref']['dyn'][1][-2] * 100)) + '%',
    showarrow=False,
    xref="x5", yref="y5",
    font=dict(family='Arial', size=14, color='rgb(248, 248, 255)'),
)

# DYN REF END
fig.add_annotation(
    x=sum(percentages['ref']['dyn'][0][0:2])+percentages['ref']['dyn'][0][-1]/2, y=0,
    text=str(round(percentages['ref']['dyn'][0][-1] * 100)) + '%',
    showarrow=False,
    xref="x5", yref="y5",
    font=dict(family='Arial', size=14, color='rgb(248, 248, 255)'),
)

fig.update_layout(height=600, width=1000)

i = 0
mini_names = [
    'Usage',
    'Curiosity',
    'Connections',
    'Creating value'
]

for an in fig['layout']['annotations']:
    if "<b>Q" in an['text']:
        an.update(font=dict(size=24))
        an.update(text=an.text + F'<br>{mini_names[i]}')
        i += 1
    
    if "<b>" in an['text'] and (not "<b>Q" in an['text']):
        an.update(x = -0.05, textangle=270, font=dict(size=20))

fig.show()

fig.write_image(f"./ief/combined.png", scale=3)



In [19]:

for q 
t, p = stats.ttest_ind(edfs['sol']['connection'], edfs['sta']['connection'], alternative='greater')

t, p

(7.423761911310085, 7.268697251999996e-13)